# Implementación del Problema de Optimización de Carteras con Computador Cuántico Adiabático

Este cuaderno describe la implementación de un problema de optimización de carteras de inversiones utilizando un computador cuántico adiabático. El enfoque adoptado es el del "Minimum Volatility Portfolio", utilizando variables binarias para la selección de activos. El objetivo principal es minimizar el riesgo total de la cartera, modelado por la fórmula:

$$\min \sum_{i,j} x_i \cdot w_{ij} \cdot x_j$$

donde $x_i$ representa la inclusión (1) o no (0) del activo $i$ en la cartera, y $w_{ij}$ es el término de covarianza entre los activos $i$ y $j$.

Además, se establece una restricción sobre el retorno esperado de la cartera, asegurando que sea al menos igual a un umbral predefinido. Esto se modela con una variable slack y se expresa como:

$$R \leq \sum_{i} x_i \cdot r_i$$

donde $r_i$ es el retorno esperado del activo $i$.

Finalmente, como condición adicional, se determina que solo se quiere invertir en la mitad de las empresas disponibles. Esto se traduce en la restricción:

$$\sum_{i} x_i = \frac{N}{2}$$

donde $N$ es el número total de activos disponibles.



In [1]:
!pip install -r "requirements_unix.txt" > /dev/null 2>&1 # Para redirigir la salida estándar y la salida de error a /dev/null

$$\min \sum_{i,j} x_i \cdot w_{ij} \cdot x_j$$


$$R \leq \sum_{i} x_i \cdot r_i$$


$$\sum_{i} x_i = \frac{N}{2}$$


In [2]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
import random
from IPython.display import display, HTML
from dadk.Optimizer import *
from dadk.SolverFactory import *
from dadk.Solution_SolutionList import *
from dadk.BinPol import *
from random import uniform
from tabulate import tabulate
from numpy import argmax
import numpy as np
from dadk.QUBOSolverCPU import *
import pandas as pd
import numpy as np

In [ ]:


# Establecemos una semilla para la reproducibilidad
np.random.seed(0)

# Creamos un DataFrame con valores aleatorios y ajustamos para tener diferentes rangos
data = {
    'EmpresaA': np.random.randint(80, 120, size=4),  # Precios moderadamente altos
    'EmpresaB': np.random.randint(50, 90, size=4),   # Precios moderados
    'EmpresaC': np.random.randint(20, 60, size=4),   # Precios bajos
    'EmpresaD': np.random.randint(100, 140, size=4)  # Precios altos
}

# Convertimos el diccionario en un DataFrame de pandas
prices = pd.DataFrame(data, index=pd.date_range(start='2024-04-20', periods=4, freq='D'))

# Mostramos el DataFrame
print(prices)


            EmpresaA  EmpresaB  EmpresaC  EmpresaD
2024-04-20        80        59        43       112
2024-04-21        83        69        26       101
2024-04-22        83        71        44       138
2024-04-23       119        86        44       139


In [3]:
# Establecemos una semilla para la reproducibilidad
np.random.seed(0)

# Creamos un DataFrame con valores aleatorios y ajustamos para tener diferentes rangos
data = {
    'EmpresaA': np.random.randint(80, 120, size=100),  # Precios moderadamente altos
    'EmpresaB': np.random.randint(50, 90, size=100),   # Precios moderados
    'EmpresaC': np.random.randint(20, 60, size=100),   # Precios bajos
    'EmpresaD': np.random.randint(100, 140, size=100)  # Precios altos
}

# Convertimos el diccionario en un DataFrame de pandas
prices = pd.DataFrame(data, index=pd.date_range(start='2024-04-20', periods=100, freq='D'))

# Mostramos el DataFrame
print(prices.head())


            EmpresaA  EmpresaB  EmpresaC  EmpresaD
2024-04-20        80        59        29       135
2024-04-21        83        50        29       119
2024-04-22        83        60        43       107
2024-04-23       119        73        23       108
2024-04-24        89        52        46       113


In [4]:
import pandas as pd
# Cargar los datos del dataset
#df = pd.read_csv('dataset_precios_acciones_DEMO.csv')

# Eliminar la primera fila porque no es un valor numérico
#prices = df.filter(like='Cierre')

# Calcular los retornos diarios y la matriz de covarianza
# La función pct_change() calcula el cambio porcentual entre filas consecutivas, días en este caso
returns = prices.pct_change().dropna()
cov_matrix = returns.cov().values


# Obtener cantidad de activos N basado en las columnas seleccionadas
n_assets = len(prices.columns)
asset_names = prices.columns.tolist()

# Función para construir el modelo QUBO
def build_qubo(cov_matrix, n_assets, factor_penalty):
    var_shape_set = VarShapeSet(BitArrayShape('x', (n_assets,)))
    BinPol.freeze_var_shape_set(var_shape_set)

    H_cuad = BinPol()
    # Construir H_cuad utilizando la matriz de covarianza
    for i in range(n_assets):
        for j in range(n_assets):
            #if i != j: # QUITAR
            w_ij = cov_matrix[i, j]
            H_cuad.add_term(w_ij, ('x', i), ('x', j))
            #print(w_ij)

    # Construir H_half para representar la restricción
    H_half = BinPol()

    for i in range(n_assets):
        H_half.add_term(1, ('x', i))
    H_half.add_term(-n_assets / 2, ())
    #H_half.add_term(-factor_penalty * (n_assets / 2), ())
    H_half.power(2)
    #H_half.multiply_by_scalar(factor_penalty)
    #H_half = H_half*factor_penalty

    # Combinar los términos para formar el QUBO
    #QUBO = BinPol()
    QUBO = H_cuad + H_half
    #QUBO.add(H_cuad)
    #QUBO.add(H_half)

    return QUBO, H_half


# Coeficiente de penalización para la restricción H_half
factor_penalty = 0  # Porque el dataset es muy pequeño
QUBO, H_half = build_qubo(cov_matrix, n_assets, factor_penalty)

# Configurar y ejecutar el solucionador QUBO
solver = QUBOSolverCPU(
    number_iterations=50000,
    number_runs=10,
    scaling_bit_precision=16,
    auto_tuning=AutoTuning.AUTO_SCALING_AND_SAMPLING
)

solution_list = solver.minimize(QUBO)
print(solution_list.solver_times)



********************************************************************************
Scaling qubo, temperature_start, temperature_end and offset_increase_rate
  factor:                        10984.00000
********************************************************************************


********************************************************************************
Effective values (including scaling factor)
  temperature_start:               28860.000
  temperature_end:                   116.300
  offset_increase_rate:             1097.000
  duration:                            0.002 sec
********************************************************************************

+--------------+----------------------------+----------------------------+----------------+
| time         | from                       | to                         | duration       |
|--------------+----------------------------+----------------------------+----------------|
| anneal       | 2024-05-05 16:55:16.482523 | 202

In [5]:
# Función para preparar y presentar el resultado
def prep_result(solution_list, H_half, silent=False):
    solution = solution_list.min_solution
    constraint_penalty = H_half.compute(solution.configuration)
    if not silent:
        print(f'Valor QUBO: {constraint_penalty}')
    return constraint_penalty, solution

# Función para reportar la solución
def report(constraint_penalty, solution, asset_names, silent=False):
    if not silent:
        if constraint_penalty < 0.1:
            print('Portfolio elegido:')
            selected_assets = [asset_names[i] for i, bit in enumerate(solution.configuration) if bit > 0.5] # Si es 1 se incluye en el portfolio y si es 0 no, por eso se pone el 0.5
            print(selected_assets)
        else:
            print("Solución inválida :(")

# Utilizar las funciones de resultado y reporte
constraint_penalty, solution = prep_result(solution_list, QUBO)
report(constraint_penalty, solution, asset_names)

Valor QUBO: 0.049463106022939574
Portfolio elegido:
['EmpresaA', 'EmpresaD']


In [ ]:
# Datos de los precios de acciones por empresa
empresaA = [80, 83, 83, 119, 89]
empresaB = [59, 50, 60, 73, 52]
empresaC = [29, 29, 43, 23, 46]
empresaD = [135, 119, 107, 108, 113]

# Calculando la media y la desviación estándar para cada empresa
media_A = np.mean(empresaA)
desviacion_A = np.std(empresaA)
media_B = np.mean(empresaB)
desviacion_B = np.std(empresaB)
media_C = np.mean(empresaC)
desviacion_C = np.std(empresaC)
media_D = np.mean(empresaD)
desviacion_D = np.std(empresaD)

print(f"Empresa A: media = {media_A}, desviación estándar = {desviacion_A}")
print(f"Empresa B: media = {media_B}, desviación estándar = {desviacion_B}")
print(f"Empresa C: media = {media_C}, desviación estándar = {desviacion_C}")
print(f"Empresa D: media = {media_D}, desviación estándar = {desviacion_D}")

Empresa A: media = 90.8, desviación estándar = 14.4
Empresa B: media = 58.8, desviación estándar = 8.084553172563094
Empresa C: media = 34.0, desviación estándar = 8.899438184514796
Empresa D: media = 116.4, desviación estándar = 10.229369482035539


Como podemos observar, la empresa D ofrece el rendimiento más alto